In [ ]:
# -------------------------------------------------------------
# Building Footprint Exposure Analysis – NC Emergency Management
# End-to-end notebook version
# -------------------------------------------------------------

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

RAW = "data_raw/microsoft_footprints.geojson"
CLEAN = "data_clean/buildings_cleaned.gpkg"
FLOOD = "hazards/flood_fema_nfhl.gpkg"
OUT = "hazards/buildings_in_floodzones.gpkg"

# -------------------------------------------------------------
# 1. LOAD & CLEAN RAW BUILDINGS
# -------------------------------------------------------------
print("Loading raw footprints...")
buildings = gpd.read_file(RAW)

# Reproject to metric CRS
buildings = buildings.to_crs(3857)

# Compute area and filter small polygons
buildings["area_m2"] = buildings.area
buildings = buildings[buildings["area_m2"] > 30]

# Fix invalid geoms
buildings["geometry"] = buildings.buffer(0)

print(f"Buildings remaining after cleaning: {len(buildings):,}")

# Save cleaned dataset
buildings.to_file(CLEAN, driver="GPKG")

# -------------------------------------------------------------
# 2. LOAD FEMA FLOOD HAZARD LAYER
# -------------------------------------------------------------
print("Loading FEMA flood hazard zones...")
flood = gpd.read_file(FLOOD)

# Filter to flood hazard categories
fema_keep = ["A", "AE", "AO", "AH", "VE", "V"]
flood = flood[flood["FLD_ZONE"].isin(fema_keep)]

# Match CRS
flood = flood.to_crs(buildings.crs)

print(f"Flood polygons: {len(flood):,}")

# -------------------------------------------------------------
# 3. SPATIAL INTERSECTION
# -------------------------------------------------------------
print("Running spatial intersection...")
intersect = gpd.sjoin(buildings, flood, how="inner", predicate="intersects")

print(f"Flooded buildings: {len(intersect):,}")

# Save intersected layer
intersect.to_file(OUT, driver="GPKG")

# -------------------------------------------------------------
# 4. SUMMARY STATISTICS
# -------------------------------------------------------------
total = len(buildings)
flooded = len(intersect)
pct = round((flooded / total) * 100, 4)

summary = pd.DataFrame({
    "Total Buildings": [total],
    "Flooded Buildings": [flooded],
    "Percent Exposed": [pct]
})

display(summary)
summary.to_csv("hazards/flood_exposure_summary.csv", index=False)

# -------------------------------------------------------------
# 5. QUICK MAP
# -------------------------------------------------------------
fig, ax = plt.subplots(figsize=(10, 10))
buildings.sample(5000).plot(ax=ax, color="lightgrey", markersize=1, alpha=0.5)
flood.boundary.plot(ax=ax, color="blue", linewidth=0.5)
intersect.plot(ax=ax, color="red", markersize=1)

plt.title("Building Footprints and FEMA Flood Zones (Demo)", fontsize=14)
plt.show()
